### 1. Test the Data Loading.

In [9]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from Model.dataloader import NutritionDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
IMAGE_DIR = '../Data/FullData/RawImages/'
BOXES = '../Data/FullData/BoundingBoxes.npy'
DATA_PATH = '../Data/FullData/train_dev_test.npy'
dataset = NutritionDataset(IMAGE_DIR, BOXES, DATA_PATH)

In [23]:
len(dataset)

36696